Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean        SD        Naive SE       MCSE      ESS
alpha 154.5965772 0.26770881 0.00423284799 0.0047908217 1000
 beta   0.9044011 0.04231353 0.00066903567 0.0005420573 1000
sigma   5.1019668 0.19184282 0.00303330135 0.0033496025 1000

Quantiles:
          2.5%        25.0%       50.0%       75.0%       97.5%   
alpha 154.0679750 154.41700000 154.595000 154.77400000 155.1280250
 beta   0.8221297   0.87665725   0.904265   0.93288775   0.9887495
sigma   4.7437200   4.96680750   5.094360   5.23065750   5.4926508



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.879 0.99626 5.12833; 155.134 0.989614 4.95238; … ; 154.863 0.853243 5.28975; 154.495 0.864436 5.36121]

Union{Missing, Float64}[154.729 0.922739 4.63957; 154.595 0.878193 5.61707; … ; 154.41 0.857143 5.14256; 154.756 0.977408 5.04832]

Union{Missing, Float64}[154.448 0.907844 5.32486; 154.558 0.934379 4.91155; … ; 155.048 0.879322 5.25822; 154.18 0.925028 4.95568]

Union{Missing, Float64}[154.532 0.837759 5.37705; 154.94 0.841972 5.23913; … ; 154.516 0.927549 5.26721; 154.938 0.895781 5.12418]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean        SD        Naive SE       MCSE      ESS
alpha 154.5965772 0.26770881 0.00423284799 0.0047908217 1000
 beta   0.9044011 0.04231353 0.00066903567 0.0005420573 1000
sigma   5.1019668 0.19184282 0.00303330135 0.0033496025 1000

Quantiles:
          2.5%        25.0%       50.0%       75.0%       97.5%   
alpha 154.0679750 154.41700000 154.595000 154.77400000 155.1280250
 beta   0.8221297   0.87665725   0.904265   0.93288775   0.9887495
sigma   4.7437200   4.96680750   5.094360   5.23065750   5.4926508



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*